In [1]:
import os
import csv
import torch
import random
import timeit
import librosa
import numpy as np
import torch.nn as nn
from scipy import signal
import torch.optim as optim
from statistics import mode
from torch.utils.data import TensorDataset, DataLoader
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2Model
from sklearn.metrics import classification_report, recall_score, accuracy_score, confusion_matrix

In [2]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-xlsr-53-french")
asr_model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-xlsr-53-french").to(DEVICE)

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53-french were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
# recola label extraction

aro_label_raw = []
val_label_raw = []

path = '/afs/inf.ed.ac.uk/user/s20/s2057508/Documents/Corpora/RECOLA/label/arousal/'
os.chdir(path)
for filename in ['P16','P17','P19','P21','P23','P25','P26','P28','P30','P34','P37','P39','P41','P42','P43','P45','P46','P48','P56','P58','P62','P64','P65']:
    with open(filename + '.csv') as f:
        file_content = csv.reader(f, delimiter=',')
        headers = next(file_content, None)
        for row in list(file_content)[1:]:
            aro_label_raw.append(int(row[-1]))
    f.close()
            
path = '/afs/inf.ed.ac.uk/user/s20/s2057508/Documents/Corpora/RECOLA/label/valence/'
os.chdir(path)
for filename in ['P16','P17','P19','P21','P23','P25','P26','P28','P30','P34','P37','P39','P41','P42','P43','P45','P46','P48','P56','P58','P62','P64','P65']:
    with open(filename + '.csv') as f:
        file_content = csv.reader(f, delimiter=',')
        headers = next(file_content, None)
        for row in list(file_content)[1:]:
            val_label_raw.append(int(row[-1]))
    f.close()
    
aro_label = []
val_label = []

# 10s
for i in range(0,690):
    if aro_label_raw[250*i:250*(i+1)].count(0) != aro_label_raw[250*i:250*(i+1)].count(1):
        aro_label.append(mode(aro_label_raw[250*i:250*(i+1)]))
    else:
        aro_label.append(1)
for i in range(0,690):
    if val_label_raw[250*i:250*(i+1)].count(0) != val_label_raw[250*i:250*(i+1)].count(1):
        val_label.append(mode(val_label_raw[250*i:250*(i+1)]))
    else:
        val_label.append(1)
        
# 5s
# for i in range(0,1380):
#     if aro_label_raw[125*i:125*(i+1)].count(0) != aro_label_raw[125*i:125*(i+1)].count(1):
#         aro_label.append(mode(aro_label_raw[125*i:125*(i+1)]))
#     else:
#         aro_label.append(1)
# for i in range(0,1380):
#     if val_label_raw[125*i:125*(i+1)].count(0) != val_label_raw[125*i:125*(i+1)].count(1):
#         val_label.append(mode(val_label_raw[125*i:125*(i+1)]))
#     else:
#         val_label.append(1)
        
print('Done!')

Done!


In [4]:
# recola feature extraction

path = '/afs/inf.ed.ac.uk/user/s20/s2057508/Documents/Corpora/RECOLA/audio/'
# path = '/afs/inf.ed.ac.uk/user/s20/s2057508/Documents/Corpora/RECOLA/audio2/'
os.chdir(path)

train_asr_hidden0 = []
train_asr_hidden12 = []
train_asr_hiddenave = []

for dirname in ['P16','P17','P19','P21','P23','P25','P26','P28','P30','P34','P37','P39','P41','P42','P43','P45','P46','P48','P56','P58','P62','P64','P65']:
    os.chdir(path + dirname)
    print(dirname)
    for filename in ['0.wav','1.wav','2.wav','3.wav','4.wav','5.wav','6.wav','7.wav','8.wav','9.wav','10.wav','11.wav','12.wav','13.wav','14.wav','15.wav','16.wav','17.wav','18.wav','19.wav','20.wav','21.wav','22.wav','23.wav','24.wav','25.wav','26.wav','27.wav','28.wav','29.wav']:
#     for filename in ['0.wav','1.wav','2.wav','3.wav','4.wav','5.wav','6.wav','7.wav','8.wav','9.wav','10.wav','11.wav','12.wav','13.wav','14.wav','15.wav','16.wav','17.wav','18.wav','19.wav','20.wav','21.wav','22.wav','23.wav','24.wav','25.wav','26.wav','27.wav','28.wav','29.wav',
#                     '30.wav','31.wav','32.wav','33.wav','34.wav','35.wav','36.wav','37.wav','38.wav','39.wav','40.wav','41.wav','42.wav','43.wav','44.wav','45.wav','46.wav','47.wav','48.wav','49.wav','50.wav','51.wav','52.wav','53.wav','54.wav','55.wav','56.wav','57.wav','58.wav','59.wav']:
        audioinput, sr = librosa.load(filename, sr=16000)
        torch.cuda.empty_cache()
                
        inputs = processor(audioinput, sampling_rate=sr, return_tensors='pt').input_values.to(DEVICE)
        hiddens = asr_model(inputs, output_hidden_states=True).hidden_states
        hidden0 = hiddens[0].detach()[0].mean(dim=0)
        hidden1 = hiddens[1].detach()[0].mean(dim=0)
        hidden2 = hiddens[2].detach()[0].mean(dim=0)
        hidden3 = hiddens[3].detach()[0].mean(dim=0)
        hidden4 = hiddens[4].detach()[0].mean(dim=0)
        hidden5 = hiddens[5].detach()[0].mean(dim=0)
        hidden6 = hiddens[6].detach()[0].mean(dim=0)
        hidden7 = hiddens[7].detach()[0].mean(dim=0)
        hidden8 = hiddens[8].detach()[0].mean(dim=0)
        hidden9 = hiddens[9].detach()[0].mean(dim=0)
        hidden10 = hiddens[10].detach()[0].mean(dim=0)
        hidden11 = hiddens[11].detach()[0].mean(dim=0)
        hidden12 = hiddens[12].detach()[0].mean(dim=0)
        hidden13 = hiddens[13].detach()[0].mean(dim=0)
        hidden14 = hiddens[14].detach()[0].mean(dim=0)
        hidden15 = hiddens[15].detach()[0].mean(dim=0)
        hidden16 = hiddens[16].detach()[0].mean(dim=0)
        hidden17 = hiddens[17].detach()[0].mean(dim=0)
        hidden18 = hiddens[18].detach()[0].mean(dim=0)
        hidden19 = hiddens[19].detach()[0].mean(dim=0)
        hidden20 = hiddens[20].detach()[0].mean(dim=0)
        hidden21 = hiddens[21].detach()[0].mean(dim=0)
        hidden22 = hiddens[22].detach()[0].mean(dim=0)
        hidden23 = hiddens[23].detach()[0].mean(dim=0)
        hidden24 = hiddens[24].detach()[0].mean(dim=0)

        hidden = (hidden0 + hidden1 + hidden2 + hidden3 + hidden4 + hidden5 + hidden6 + hidden7 + hidden8 + hidden9 + hidden10 + hidden11
                  + hidden12 + hidden13 + hidden14 + hidden15 + hidden16 + hidden17 + hidden18 + hidden19 + hidden20 + hidden21 + hidden22 + hidden23 + hidden24) / 25.0

        train_asr_hidden0.append(hidden0.view(1, 1024).cpu().numpy())
        train_asr_hidden12.append(hidden12.view(1, 1024).cpu().numpy())
        train_asr_hiddenave.append(hidden.view(1, 1024).cpu().numpy())
        
        torch.cuda.empty_cache()
        
print('Done!')

P16
P17
P19
P21
P23
P25
P26
P28
P30
P34
P37
P39
P41
P42
P43
P45
P46
P48
P56
P58
P62
P64
P65
Done!


In [5]:
# sspnet feature & label preparation
path = "/afs/inf.ed.ac.uk/user/s20/s2057508/Documents/Corpora/SSPNet/Audio"
os.chdir(path)

valid_ex = []
valid_ag = []
valid_co = []
valid_ne = []
valid_op = []
test_ex = []
test_ag = []
test_co = []
test_ne = []
test_op = []

valid_asr_hidden0 = []
valid_asr_hidden12 = []
valid_asr_hiddenave = []

test_asr_hidden0 = []
test_asr_hidden12 = []
test_asr_hiddenave = []

# valid feature
with open('Scores.csv', 'r') as f:
    file_content = csv.reader(f, delimiter=',')
    headers = next(file_content, None)
    for row in list(file_content)[:512]:
        audioinput, sr = librosa.load(row[0] + '.wav', sr=16000)
#         audioinput = audioinput[2:]
#         if len(audioinput) < 160000:
#             audioinput = np.pad(audioinput, (0, 160000-len(audioinput)), 'constant', constant_values=0)
#             audioinput = torch.from_numpy(audioinput)
        
        torch.cuda.empty_cache()
        
        inputs = processor(audioinput, sampling_rate=sr, return_tensors='pt').input_values.to(DEVICE)
        hiddens = asr_model(inputs, output_hidden_states=True).hidden_states
        hidden0 = hiddens[0].detach()[0].mean(dim=0)
        hidden1 = hiddens[1].detach()[0].mean(dim=0)
        hidden2 = hiddens[2].detach()[0].mean(dim=0)
        hidden3 = hiddens[3].detach()[0].mean(dim=0)
        hidden4 = hiddens[4].detach()[0].mean(dim=0)
        hidden5 = hiddens[5].detach()[0].mean(dim=0)
        hidden6 = hiddens[6].detach()[0].mean(dim=0)
        hidden7 = hiddens[7].detach()[0].mean(dim=0)
        hidden8 = hiddens[8].detach()[0].mean(dim=0)
        hidden9 = hiddens[9].detach()[0].mean(dim=0)
        hidden10 = hiddens[10].detach()[0].mean(dim=0)
        hidden11 = hiddens[11].detach()[0].mean(dim=0)
        hidden12 = hiddens[12].detach()[0].mean(dim=0)
        hidden13 = hiddens[13].detach()[0].mean(dim=0)
        hidden14 = hiddens[14].detach()[0].mean(dim=0)
        hidden15 = hiddens[15].detach()[0].mean(dim=0)
        hidden16 = hiddens[16].detach()[0].mean(dim=0)
        hidden17 = hiddens[17].detach()[0].mean(dim=0)
        hidden18 = hiddens[18].detach()[0].mean(dim=0)
        hidden19 = hiddens[19].detach()[0].mean(dim=0)
        hidden20 = hiddens[20].detach()[0].mean(dim=0)
        hidden21 = hiddens[21].detach()[0].mean(dim=0)
        hidden22 = hiddens[22].detach()[0].mean(dim=0)
        hidden23 = hiddens[23].detach()[0].mean(dim=0)
        hidden24 = hiddens[24].detach()[0].mean(dim=0)

        hidden = (hidden0 + hidden1 + hidden2 + hidden3 + hidden4 + hidden5 + hidden6 + hidden7 + hidden8 + hidden9 + hidden10 + hidden11
                  + hidden12 + hidden13 + hidden14 + hidden15 + hidden16 + hidden17 + hidden18 + hidden19 + hidden20 + hidden21 + hidden22 + hidden23 + hidden24) / 25.0

        valid_asr_hidden0.append(hidden0.view(1, 1024).cpu().numpy())
        valid_asr_hidden12.append(hidden6.view(1, 1024).cpu().numpy())
        valid_asr_hiddenave.append(hidden.view(1, 1024).cpu().numpy())
        
        torch.cuda.empty_cache()
        
        valid_ex.append(int(row[1]))
        valid_ag.append(int(row[2]))
        valid_co.append(int(row[3]))
        valid_ne.append(int(row[4]))
        valid_op.append(int(row[5]))
        
f.close()
        
# test feature
with open('Scores.csv', 'r') as f:
    file_content = csv.reader(f, delimiter=',')
    headers = next(file_content, None)
    for row in list(file_content)[512:]:
        audioinput, sr = librosa.load(row[0] + '.wav', sr=16000)
#         audioinput = audioinput[2:]
#         if len(audioinput) < 160000:
#             audioinput = np.pad(audioinput, (0, 160000-len(audioinput)), 'constant', constant_values=0)
#             audioinput = torch.from_numpy(audioinput)
        
        torch.cuda.empty_cache()
        
        inputs = processor(audioinput, sampling_rate=sr, return_tensors='pt').input_values.to(DEVICE)
        hiddens = asr_model(inputs, output_hidden_states=True).hidden_states
        hidden0 = hiddens[0].detach()[0].mean(dim=0)
        hidden1 = hiddens[1].detach()[0].mean(dim=0)
        hidden2 = hiddens[2].detach()[0].mean(dim=0)
        hidden3 = hiddens[3].detach()[0].mean(dim=0)
        hidden4 = hiddens[4].detach()[0].mean(dim=0)
        hidden5 = hiddens[5].detach()[0].mean(dim=0)
        hidden6 = hiddens[6].detach()[0].mean(dim=0)
        hidden7 = hiddens[7].detach()[0].mean(dim=0)
        hidden8 = hiddens[8].detach()[0].mean(dim=0)
        hidden9 = hiddens[9].detach()[0].mean(dim=0)
        hidden10 = hiddens[10].detach()[0].mean(dim=0)
        hidden11 = hiddens[11].detach()[0].mean(dim=0)
        hidden12 = hiddens[12].detach()[0].mean(dim=0)
        hidden13 = hiddens[13].detach()[0].mean(dim=0)
        hidden14 = hiddens[14].detach()[0].mean(dim=0)
        hidden15 = hiddens[15].detach()[0].mean(dim=0)
        hidden16 = hiddens[16].detach()[0].mean(dim=0)
        hidden17 = hiddens[17].detach()[0].mean(dim=0)
        hidden18 = hiddens[18].detach()[0].mean(dim=0)
        hidden19 = hiddens[19].detach()[0].mean(dim=0)
        hidden20 = hiddens[20].detach()[0].mean(dim=0)
        hidden21 = hiddens[21].detach()[0].mean(dim=0)
        hidden22 = hiddens[22].detach()[0].mean(dim=0)
        hidden23 = hiddens[23].detach()[0].mean(dim=0)
        hidden24 = hiddens[24].detach()[0].mean(dim=0)

        hidden = (hidden0 + hidden1 + hidden2 + hidden3 + hidden4 + hidden5 + hidden6 + hidden7 + hidden8 + hidden9 + hidden10 + hidden11
                  + hidden12 + hidden13 + hidden14 + hidden15 + hidden16 + hidden17 + hidden18 + hidden19 + hidden20 + hidden21 + hidden22 + hidden23 + hidden24) / 25.0

        test_asr_hidden0.append(hidden0.view(1, 1024).cpu().numpy())
        test_asr_hidden12.append(hidden6.view(1, 1024).cpu().numpy())
        test_asr_hiddenave.append(hidden.view(1, 1024).cpu().numpy())
        
        torch.cuda.empty_cache()
        
        test_ex.append(int(row[1]))
        test_ag.append(int(row[2]))
        test_co.append(int(row[3]))
        test_ne.append(int(row[4]))
        test_op.append(int(row[5]))

f.close()

print('Done!')

Done!


In [6]:
train_asr_hidden0 = np.array(train_asr_hidden0)
train_asr_hidden12 = np.array(train_asr_hidden12)
train_asr_hiddenave = np.array(train_asr_hiddenave)
train_asr_hidden0 = torch.from_numpy(train_asr_hidden0).to(DEVICE)
train_asr_hidden12 = torch.from_numpy(train_asr_hidden12).to(DEVICE)
train_asr_hiddenave = torch.from_numpy(train_asr_hiddenave).to(DEVICE)
valid_asr_hidden0 = np.array(valid_asr_hidden0)
valid_asr_hidden12 = np.array(valid_asr_hidden12)
valid_asr_hiddenave = np.array(valid_asr_hiddenave)
valid_asr_hidden0 = torch.from_numpy(valid_asr_hidden0).to(DEVICE)
valid_asr_hidden12 = torch.from_numpy(valid_asr_hidden12).to(DEVICE)
valid_asr_hiddenave = torch.from_numpy(valid_asr_hiddenave).to(DEVICE)
test_asr_hidden0 = np.array(test_asr_hidden0)
test_asr_hidden12 = np.array(test_asr_hidden12)
test_asr_hiddenave = np.array(test_asr_hiddenave)
test_asr_hidden0 = torch.from_numpy(test_asr_hidden0).to(DEVICE)
test_asr_hidden12 = torch.from_numpy(test_asr_hidden12).to(DEVICE)
test_asr_hiddenave = torch.from_numpy(test_asr_hiddenave).to(DEVICE)
aro_label = torch.tensor(aro_label).to(DEVICE)
val_label = torch.tensor(val_label).to(DEVICE)
valid_ex = torch.tensor(valid_ex).to(DEVICE)
valid_ag = torch.tensor(valid_ag).to(DEVICE)
valid_co = torch.tensor(valid_co).to(DEVICE)
valid_ne = torch.tensor(valid_ne).to(DEVICE)
valid_op = torch.tensor(valid_op).to(DEVICE)
test_ex = torch.tensor(test_ex).to(DEVICE)
test_ag = torch.tensor(test_ag).to(DEVICE)
test_co = torch.tensor(test_co).to(DEVICE)
test_ne = torch.tensor(test_ne).to(DEVICE)
test_op = torch.tensor(test_op).to(DEVICE)

In [7]:
print(train_asr_hidden0.size(), valid_asr_hidden0.size(), test_asr_hidden0.size())

torch.Size([690, 1, 1024]) torch.Size([512, 1, 1024]) torch.Size([128, 1, 1024])


In [8]:
trainset = TensorDataset(train_asr_hidden0, train_asr_hidden12, train_asr_hiddenave, aro_label, val_label)
traindata = DataLoader(dataset=trainset, batch_size=64, shuffle=True)
validset = TensorDataset(valid_asr_hidden0, valid_asr_hidden12, valid_asr_hiddenave, valid_ex, valid_ag, valid_co, valid_ne, valid_op)
validdata = DataLoader(dataset=validset, batch_size=64, shuffle=True)
testset = TensorDataset(test_asr_hidden0, test_asr_hidden12, test_asr_hiddenave, test_ex, test_ag, test_co, test_ne, test_op)
testdata = DataLoader(dataset=testset, batch_size=64, shuffle=False)

In [21]:
# MLP
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet,self).__init__()
        self.flat = nn.Flatten()
        self.dense1 = nn.Linear(1024, 128)
        self.drop = nn.Dropout(0.5)
        self.dense = nn.Linear(128, 16)
        self.acti = nn.ReLU()
        self.out1 = nn.Linear(16, 2)
        self.out2 = nn.Linear(16, 2)

    def forward(self, x):
        x = self.flat(x)
        x = self.dense1(x)
        x = self.drop(x)
        x = self.dense(x)
        x = self.drop(x)
        res = self.acti(x)
        aro = self.out1(res)
#         val = self.out2(res)
        return aro

In [55]:
# train w2v training (fine-tuning)

# training

for k in [10,20,30,40,50,60,70,80,90,100]:
    
    torch.manual_seed(1)

    model = NeuralNet().to(DEVICE)
    params = model.parameters()
    params = list(asr_model.parameters()) + list(model.parameters())
#     optimizer = torch.optim.AdamW(params, lr=1e-4, weight_decay=1e-5)
    optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, params), lr=1e-4, weight_decay=1e-5)
    func = nn.CrossEntropyLoss()

    for epoch in range(k):
        
        model.train()
        asr_model.train()
        
        unfreeze = ['classifier', 'pooler', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']
        for name, param in asr_model.named_parameters():
            if any([ll in name for ll in unfreeze]):
                param.requires_grad = True
            else:
                param.requires_grad = False
        
        torch.cuda.empty_cache()

        predictions_train = []
        labels_train = []
        loss_train = []

        for feats2, feats3, feats, labels2, labels in traindata:

    # loss
            preds = model(feats)
            train_loss = func(preds, labels)
            loss_train.append(train_loss.detach().cpu().numpy())

            for i in preds:
                predictions_train.append(i.detach().cpu().numpy())

            for i in labels:
                labels_train.append(i.detach().cpu().numpy())

    # backprop
            optimizer.zero_grad()
            torch.nn.utils.clip_grad_norm_(params, 1.0)
            train_loss.backward()
            optimizer.step()
            torch.cuda.empty_cache()

    # compute performance for each epoch
        predictions_train = np.array(predictions_train)
        labels_train = np.array(labels_train)
        predictions_train = [np.argmax(p) for p in predictions_train]
        acc_train = recall_score(labels_train, predictions_train, average='macro')
        loss_train = sum(loss_train) / len(loss_train)


# fine-tuning

    torch.manual_seed(1)

    best = []

    for epoch in range(100):
        
        torch.cuda.empty_cache()

        predictions_valid = []
        predictions_test = []
        labels_valid = []
        labels_test = []
        loss_valid = []
        loss_test = []


        for feats2, feats3, feats, labels2, labels3, labels4, labels5, labels in validdata:
            preds = model(feats)
            valid_loss = func(preds, labels)
            loss_valid.append(valid_loss.detach().cpu().numpy())

            for i in preds:
                predictions_valid.append(i.detach().cpu().numpy())
            for i in labels:
                labels_valid.append(i.detach().cpu().numpy())

        # backprop
            optimizer.zero_grad()
            torch.nn.utils.clip_grad_norm_(params, 1.0)
            valid_loss.backward()
            optimizer.step()
            torch.cuda.empty_cache()

        # test
        
        model.eval()
        asr_model.eval()
        
        for feats2, feats3, feats, labels2, labels3, labels4, labels5, labels in testdata:
            preds = model(feats)
            test_loss = func(preds, labels)
            loss_test.append(test_loss.detach().cpu().numpy())

            for i in preds:
                predictions_test.append(i.detach().cpu().numpy())
            for i in labels:
                labels_test.append(i.detach().cpu().numpy())

        # compute performance for each epoch
        predictions_valid = np.array(predictions_valid)
        predictions_test = np.array(predictions_test)
        labels_valid = np.array(labels_valid)
        labels_test = np.array(labels_test)
        predictions_valid = [np.argmax(p) for p in predictions_valid]
        predictions_test = [np.argmax(p) for p in predictions_test]

    #     print(predictions_test)    

        acc_valid = recall_score(labels_valid, predictions_valid, average='macro')
        acc_test = recall_score(labels_test, predictions_test, average='macro')
        loss_valid = sum(loss_valid) / len(loss_valid)
        loss_test = sum(loss_test) / len(loss_test)

        best.append(acc_test)


    print(k, round(max(best), 4))

10 0.5
20 0.5
30 0.5
40 0.5
50 0.5
60 0.5
70 0.5
80 0.5
90 0.5
100 0.5


In [17]:
# train

# training

for k in [10,20,30,40,50,60,70,80,90,100]:
    
    torch.manual_seed(1)

    model = NeuralNet().to(DEVICE)
    params = model.parameters()
    optimizer = torch.optim.AdamW(params, lr=1e-4, weight_decay=1e-5)
    func = nn.CrossEntropyLoss()

    for epoch in range(k):
        torch.cuda.empty_cache()

        predictions_train = []
        labels_train = []
        loss_train = []

        for feats2, feats, feats3, labels, labels2 in traindata:

    # loss
            preds = model(feats)
            train_loss = func(preds, labels)
            loss_train.append(train_loss.detach().cpu().numpy())

            for i in preds:
                predictions_train.append(i.detach().cpu().numpy())

            for i in labels:
                labels_train.append(i.detach().cpu().numpy())

    # backprop
            optimizer.zero_grad()
            torch.nn.utils.clip_grad_norm_(params, 1.0)
            train_loss.backward()
            optimizer.step()
            torch.cuda.empty_cache()

    # compute performance for each epoch
        predictions_train = np.array(predictions_train)
        labels_train = np.array(labels_train)
        predictions_train = [np.argmax(p) for p in predictions_train]
        acc_train = recall_score(labels_train, predictions_train, average='macro')
        loss_train = sum(loss_train) / len(loss_train)


# fine-tuning

    torch.manual_seed(1)

    best = []
    
#     model.eval()

    for epoch in range(100):
        # valid
        
        torch.cuda.empty_cache()

        predictions_valid = []
        predictions_test = []
        labels_valid = []
        labels_test = []
        loss_valid = []
        loss_test = []


        for feats2, feats, feats3, labels, labels2, labels3, labels4, labels5 in validdata:
            preds = model(feats)
            valid_loss = func(preds, labels)
            loss_valid.append(valid_loss.detach().cpu().numpy())

            for i in preds:
                predictions_valid.append(i.detach().cpu().numpy())
            for i in labels:
                labels_valid.append(i.detach().cpu().numpy())

        # backprop
            optimizer.zero_grad()
            torch.nn.utils.clip_grad_norm_(params, 1.0)
            valid_loss.backward()
            optimizer.step()
            torch.cuda.empty_cache()

        # test
        
        model.eval()
        for feats2, feats, feats3, labels, labels2, labels3, labels4, labels5 in testdata:
            preds = model(feats)
            test_loss = func(preds, labels)
            loss_test.append(test_loss.detach().cpu().numpy())

            for i in preds:
                predictions_test.append(i.detach().cpu().numpy())
            for i in labels:
                labels_test.append(i.detach().cpu().numpy())

        # compute performance for each epoch
        predictions_valid = np.array(predictions_valid)
        predictions_test = np.array(predictions_test)
        labels_valid = np.array(labels_valid)
        labels_test = np.array(labels_test)
        predictions_valid = [np.argmax(p) for p in predictions_valid]
        predictions_test = [np.argmax(p) for p in predictions_test]

    #     print(predictions_test)    

        acc_valid = recall_score(labels_valid, predictions_valid, average='macro')
        acc_test = recall_score(labels_test, predictions_test, average='macro')
        loss_valid = sum(loss_valid) / len(loss_valid)
        loss_test = sum(loss_test) / len(loss_test)

        best.append(acc_test)


    print(k, round(max(best), 4))

10 0.671
20 0.6602
30 0.6541
40 0.6614
50 0.6577
60 0.6541
70 0.6349
80 0.6613
90 0.6408
100 0.6347


In [19]:
# train-multitask w2v training (fine-tuning)

# training

for k in [10,20,30,40,50,60,70,80,90,100]:
    
    torch.manual_seed(1)

    model = NeuralNet().to(DEVICE)
    params = model.parameters()
    optimizer = torch.optim.AdamW(params, lr=1e-4, weight_decay=1e-5)
    func = nn.CrossEntropyLoss()

    for epoch in range(k):
        torch.cuda.empty_cache()

        model.train()
        asr_model.train()

        predictions1_train = []
        predictions2_train = []
        labels1_train = []
        labels2_train = []
        loss_train = []

        for feats1, feats2, feats3, labels1, labels2 in traindata:

    # loss
            preds1, preds2 = model(feats1)
            train_loss1 = func(preds1, labels1)
            train_loss2 = func(preds2, labels2)
            train_loss = train_loss1 + train_loss2
            loss_train.append(train_loss.detach().cpu().numpy())

            for i in preds1:
                predictions1_train.append(i.detach().cpu().numpy())                
            for i in preds2:
                predictions2_train.append(i.detach().cpu().numpy())

            for i in labels1:
                labels1_train.append(i.detach().cpu().numpy())
            for i in labels2:
                labels2_train.append(i.detach().cpu().numpy())
                
    # backprop
            optimizer.zero_grad()
            torch.nn.utils.clip_grad_norm_(params, 1.0)
            train_loss.backward()
            optimizer.step()
            torch.cuda.empty_cache()

    # compute performance for each epoch
        predictions1_train = np.array(predictions1_train)
        predictions2_train = np.array(predictions2_train)
        labels1_train = np.array(labels1_train)
        labels2_train = np.array(labels2_train)
        predictions1_train = [np.argmax(p) for p in predictions1_train]
        predictions2_train = [np.argmax(p) for p in predictions2_train]
        acc1_train = recall_score(labels1_train, predictions1_train, average='macro')
        acc2_train = recall_score(labels2_train, predictions2_train, average='macro')
        loss_train = sum(loss_train) / len(loss_train)


# fine-tuning

#     torch.manual_seed(1)

    best1 = []
    best2 = []
    
    model.eval()
    asr_model.eval()

    for epoch in range(100):
        # valid
        
        torch.cuda.empty_cache()

        predictions1_valid = []
        predictions2_valid = []
        predictions1_test = []
        predictions2_test = []
        labels1_valid = []
        labels2_valid = []
        labels1_test = []
        labels2_test = []
        loss_valid = []
        loss_test = []


        for feats1, feats2, feats3, labels1, labels3, labels4, labels5, labels2 in validdata:
            preds1, preds2 = model(feats1)
            valid_loss1 = func(preds1, labels1)
            valid_loss2 = func(preds2, labels2)
            valid_loss = valid_loss1 + valid_loss2
            loss_valid.append(valid_loss.detach().cpu().numpy())

            for i in preds1:
                predictions1_valid.append(i.detach().cpu().numpy())
            for i in preds2:
                predictions2_valid.append(i.detach().cpu().numpy())
            for i in labels1:
                labels1_valid.append(i.detach().cpu().numpy())
            for i in labels2:
                labels2_valid.append(i.detach().cpu().numpy())
                
        # backprop
            optimizer.zero_grad()
            torch.nn.utils.clip_grad_norm_(params, 1.0)
            valid_loss.backward()
            optimizer.step()
            torch.cuda.empty_cache()

        # test
        
        for feats1, feats2, feats3, labels1, labels3, labels4, labels5, labels2 in testdata:
            preds1, preds2 = model(feats1)
            test_loss1 = func(preds1, labels1)
            test_loss2 = func(preds2, labels2)
            test_loss = test_loss1 + test_loss2
            loss_test.append(test_loss.detach().cpu().numpy())

            for i in preds1:
                predictions1_test.append(i.detach().cpu().numpy())
            for i in preds2:
                predictions2_test.append(i.detach().cpu().numpy())
            for i in labels1:
                labels1_test.append(i.detach().cpu().numpy())
            for i in labels2:
                labels2_test.append(i.detach().cpu().numpy())

        # compute performance for each epoch
        predictions1_valid = np.array(predictions1_valid)
        predictions2_valid = np.array(predictions2_valid)
        predictions1_test = np.array(predictions1_test)
        predictions2_test = np.array(predictions2_test)
        labels1_valid = np.array(labels1_valid)
        labels2_valid = np.array(labels2_valid)
        labels1_test = np.array(labels1_test)
        labels2_test = np.array(labels2_test)
        predictions1_valid = [np.argmax(p) for p in predictions1_valid]
        predictions2_valid = [np.argmax(p) for p in predictions2_valid]
        predictions1_test = [np.argmax(p) for p in predictions1_test]
        predictions2_test = [np.argmax(p) for p in predictions2_test]

    #     print(predictions_test)    

        acc1_valid = recall_score(labels1_valid, predictions1_valid, average='macro')
        acc2_valid = recall_score(labels2_valid, predictions2_valid, average='macro')
        acc1_test = recall_score(labels1_test, predictions1_test, average='macro')
        acc2_test = recall_score(labels2_test, predictions2_test, average='macro')
        loss_valid = sum(loss_valid) / len(loss_valid)
        loss_test = sum(loss_test) / len(loss_test)

        best1.append(acc1_test)
        best2.append(acc2_test)

    print(k, round(max(best1), 4), round(max(best2), 4))

10 0.5733 0.5
20 0.6503 0.5
30 0.5075 0.5365
40 0.6371 0.5
50 0.5122 0.5565
60 0.6382 0.558


KeyboardInterrupt: 

In [ ]:
# train-multitask

# training

for k in [10,20,30,40,50,60,70,80,90,100]:
    
#     torch.manual_seed(1)

    model = NeuralNet().to(DEVICE)
    params = model.parameters()
    optimizer = torch.optim.AdamW(params, lr=1e-4, weight_decay=1e-5)
    func = nn.CrossEntropyLoss()

    for epoch in range(k):
        torch.cuda.empty_cache()

        predictions1_train = []
        predictions2_train = []
        labels1_train = []
        labels2_train = []
        loss_train = []

        for feats1, feats2, feats3, labels1, labels2 in traindata:

    # loss
            preds1, preds2 = model(feats1)
            train_loss1 = func(preds1, labels1)
            train_loss2 = func(preds2, labels2)
            train_loss = train_loss1 + train_loss2
            loss_train.append(train_loss.detach().cpu().numpy())

            for i in preds1:
                predictions1_train.append(i.detach().cpu().numpy())                
            for i in preds2:
                predictions2_train.append(i.detach().cpu().numpy())

            for i in labels1:
                labels1_train.append(i.detach().cpu().numpy())
            for i in labels2:
                labels2_train.append(i.detach().cpu().numpy())
                
    # backprop
            optimizer.zero_grad()
            torch.nn.utils.clip_grad_norm_(params, 1.0)
            train_loss.backward()
            optimizer.step()
            torch.cuda.empty_cache()

    # compute performance for each epoch
        predictions1_train = np.array(predictions1_train)
        predictions2_train = np.array(predictions2_train)
        labels1_train = np.array(labels1_train)
        labels2_train = np.array(labels2_train)
        predictions1_train = [np.argmax(p) for p in predictions1_train]
        predictions2_train = [np.argmax(p) for p in predictions2_train]
        acc1_train = recall_score(labels1_train, predictions1_train, average='macro')
        acc2_train = recall_score(labels2_train, predictions2_train, average='macro')
        loss_train = sum(loss_train) / len(loss_train)


# fine-tuning

#     torch.manual_seed(1)

    best1 = []
    best2 = []
    
    model.eval()

    for epoch in range(100):
        # valid
        
        torch.cuda.empty_cache()

        predictions1_valid = []
        predictions2_valid = []
        predictions1_test = []
        predictions2_test = []
        labels1_valid = []
        labels2_valid = []
        labels1_test = []
        labels2_test = []
        loss_valid = []
        loss_test = []


        for feats1, feats2, feats3, labels1, labels3, labels4, labels5, labels2 in validdata:
            preds1, preds2 = model(feats1)
            valid_loss1 = func(preds1, labels1)
            valid_loss2 = func(preds2, labels2)
            valid_loss = valid_loss1 + valid_loss2
            loss_valid.append(valid_loss.detach().cpu().numpy())

            for i in preds1:
                predictions1_valid.append(i.detach().cpu().numpy())
            for i in preds2:
                predictions2_valid.append(i.detach().cpu().numpy())
            for i in labels1:
                labels1_valid.append(i.detach().cpu().numpy())
            for i in labels2:
                labels2_valid.append(i.detach().cpu().numpy())
                
        # backprop
            optimizer.zero_grad()
            torch.nn.utils.clip_grad_norm_(params, 1.0)
            valid_loss.backward()
            optimizer.step()
            torch.cuda.empty_cache()

        # test
        
        for feats1, feats2, feats3, labels1, labels3, labels4, labels5, labels2 in testdata:
            preds1, preds2 = model(feats1)
            test_loss1 = func(preds1, labels1)
            test_loss2 = func(preds2, labels2)
            test_loss = test_loss1 + test_loss2
            loss_test.append(test_loss.detach().cpu().numpy())

            for i in preds1:
                predictions1_test.append(i.detach().cpu().numpy())
            for i in preds2:
                predictions2_test.append(i.detach().cpu().numpy())
            for i in labels1:
                labels1_test.append(i.detach().cpu().numpy())
            for i in labels2:
                labels2_test.append(i.detach().cpu().numpy())

        # compute performance for each epoch
        predictions1_valid = np.array(predictions1_valid)
        predictions2_valid = np.array(predictions2_valid)
        predictions1_test = np.array(predictions1_test)
        predictions2_test = np.array(predictions2_test)
        labels1_valid = np.array(labels1_valid)
        labels2_valid = np.array(labels2_valid)
        labels1_test = np.array(labels1_test)
        labels2_test = np.array(labels2_test)
        predictions1_valid = [np.argmax(p) for p in predictions1_valid]
        predictions2_valid = [np.argmax(p) for p in predictions2_valid]
        predictions1_test = [np.argmax(p) for p in predictions1_test]
        predictions2_test = [np.argmax(p) for p in predictions2_test]

    #     print(predictions_test)    

        acc1_valid = recall_score(labels1_valid, predictions1_valid, average='macro')
        acc2_valid = recall_score(labels2_valid, predictions2_valid, average='macro')
        acc1_test = recall_score(labels1_test, predictions1_test, average='macro')
        acc2_test = recall_score(labels2_test, predictions2_test, average='macro')
        loss_valid = sum(loss_valid) / len(loss_valid)
        loss_test = sum(loss_test) / len(loss_test)

        best1.append(acc1_test)
        best2.append(acc2_test)

    print(k, round(max(best1), 4), round(max(best2), 4))